In [97]:
# Importação das bibliotecas
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score

In [ ]:
# Carregar os dados preparados
x_treino = np.load('x_treino.npy')  # Carrega as imagens de treinamento
y_treino = np.load('y_treino.npy')  # Carrega os rótulos de treinamento
z_treino = np.load('z_treino.npy')

x_validacao = np.load('x_validacao.npy')  # Carrega as imagens de validação
y_validacao = np.load('y_validacao.npy')  # Carrega os rótulos de validação
z_validacao = np.load('z_validacao.npy')

x_teste = np.load('x_teste.npy')  # Carrega as imagens de teste
y_teste = np.load('y_teste.npy')  # Carrega os rótulos de teste
z_teste = np.load('z_teste.npy')

print(np.shape(x_treino))
print(np.shape(x_validacao))
print(np.shape(x_teste))
print(y_teste)

In [99]:
x_treino = x_treino.reshape(-1, 64, 64, 1)
x_validacao = x_validacao.reshape(-1, 64, 64, 1)
x_teste = x_teste.reshape(-1, 64, 64, 1)

In [ ]:
letras_mapping = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "I": 9, "L": 12, "M": 13, "N": 14, "O": 15, "P": 16, "Q": 17, "R": 18, "S": 19, "T": 20, "U": 21, "V": 22, "W": 23, "Y": 25}

letras_indices = {letra: idx for idx, letra in enumerate(letras_mapping.keys())}
y_treino = to_categorical([letras_indices[letra] for letra in y_treino])
y_validacao = to_categorical([letras_indices[letra] for letra in y_validacao])
y_teste = to_categorical([letras_indices[letra] for letra in y_teste])
print(y_treino)
print(y_validacao)
print(y_teste)

In [101]:
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(letras_mapping), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
cnn_model = create_cnn_model()
history = cnn_model.fit(
    x_treino, y_treino,
    validation_data=(x_validacao, y_validacao),
    epochs=10,
    batch_size=32
)

In [ ]:
test_loss, test_acc = cnn_model.evaluate(x_teste, y_teste)
print(f"Acurácia no teste: {test_acc:.2f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Erro de Treinamento')
plt.plot(history.history['val_loss'], label='Erro de Teste')
plt.legend()
plt.title('Erro por Época')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Teste')
plt.legend()
plt.title('Acurácia por Época')
plt.show()

In [ ]:
y_pred = cnn_model.predict(x_teste).argmax(axis=1)
y_true = y_teste.argmax(axis=1)
conf_matrix = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=letras_mapping.keys())
disp.plot(cmap='Blues')
plt.xlabel('Rótulos previstos')  
plt.ylabel('Rótulos verdadeiros')  
plt.title('Matriz de Confusão - CNN Padrão')
plt.show()

In [109]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential([
        Conv2D(hp.Int('filters', 32, 128, step=32), (3, 3), activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D((2, 2)),
        Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)),
        Conv2D(hp.Int('filters2', 32, 128, step=32), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(hp.Int('dense_units', 64, 256, step=64), activation='relu'),
        Dropout(hp.Float('dense_dropout', 0.2, 0.5, step=0.1)),
        Dense(len(letras_mapping), activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,  
    directory='my_dir',
    project_name='cnn_tuner'
)

In [113]:
tuner.search(x_treino, y_treino, validation_data=(x_validacao, y_validacao), epochs=5, batch_size=32)

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print(f"Melhores Hiperparâmetros: {best_hps.values}")

best_model = tuner.hypermodel.build(best_hps)
history_optimized = best_model.fit(x_treino, y_treino, validation_data=(x_validacao, y_validacao), epochs=10, batch_size=32)
test_loss_optimized, test_acc_optimized = best_model.evaluate(x_teste, y_teste)
print(f"Acurácia otimizada no teste: {test_acc_optimized:.2f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_optimized.history['loss'], label='Erro de Treinamento')
plt.plot(history_optimized.history['val_loss'], label='Erro de Teste')
plt.legend()
plt.title('Erro por Época - Modelo Otimizado')

plt.subplot(1, 2, 2)
plt.plot(history_optimized.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history_optimized.history['val_accuracy'], label='Acurácia de Teste')
plt.legend()
plt.title('Acurácia por Época - Modelo Otimizado')
plt.show()


In [ ]:
y_pred_optimized = best_model.predict(x_teste).argmax(axis=1)
conf_matrix_optimized = confusion_matrix(y_true, y_pred_optimized)
disp = ConfusionMatrixDisplay(conf_matrix_optimized, display_labels=letras_mapping.keys())
disp.plot(cmap='Blues')
plt.xlabel('Rótulos previstos')  
plt.ylabel('Rótulos verdadeiros')  
plt.title('Matriz de Confusão - Modelo Otimizado')
plt.show()

In [ ]:
x_teste_ruidosa = x_teste + np.random.normal(0, 0.1, x_teste.shape)

x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1).astype(np.float32)

print("Forma dos dados ruidosos:", x_teste_ruidosa.shape)
print(y_teste)

loss, accuracy = cnn_model.evaluate(x_teste_ruidosa, y_teste)

print("Acurácia Ruidosa:", accuracy)

In [ ]:
x_teste_ruidosa = x_teste + np.random.normal(0, 0.1, x_teste.shape)

x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1).astype(np.float32)


print("Forma dos dados ruidosos:", x_teste_ruidosa.shape)
print(y_teste)


loss, accuracy = best_model.evaluate(x_teste_ruidosa, y_teste)

print("Acurácia Ruidosa:", accuracy)